In [2]:
import pandas as pd
import geopandas as gpd

In [40]:
agebs = gpd.read_file("../datos/agebs_cdmx/09a.shp")
datag = pd.read_csv("../datos/agebs_cdmx/RESAGEBURB_09CSV20.csv")

C:\Users\irvin\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (178) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [41]:
datag[:3]

,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,AGEB,MZA,POBTOT,POBFEM,...,VPH_TELEF,VPH_CEL,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,VPH_SINRTV,VPH_SINLTC,VPH_SINCINT,VPH_SINTIC
0,9,Ciudad de México,0,Total de la entidad Ciudad de México,0,Total de la entidad,0000,0,9209944,4805017,...,1898265,2536523,2084156,1290811,957162,568827,46172,77272,561128,10528
1,9,Ciudad de México,2,Azcapotzalco,0,Total del municipio,0000,0,432205,227255,...,96128,123961,105899,66399,50965,31801,1661,2869,22687,322
2,9,Ciudad de México,2,Azcapotzalco,1,Total de la localidad urbana,0000,0,432205,227255,...,96128,123961,105899,66399,50965,31801,1661,2869,22687,322


In [42]:
datag["ENTIDAD"] = datag["ENTIDAD"].astype(str).str.zfill(2)
datag["MUN"] = datag["MUN"].astype(str).str.zfill(3)
datag["LOC"] = datag["LOC"].astype(str).str.zfill(4)
datag["AGEB"] = datag["AGEB"].astype(str).str.zfill(4)
datag["MZA"] = datag["MZA"].astype(str).str.zfill(3)

In [43]:
datagf = datag[(datag["ENTIDAD"] != "00") & (datag["MUN"] != "000") & (datag["LOC"] != "0000") & (datag["AGEB"] != "0000") & (datag["MZA"] == "000")]

In [44]:
datagf["CVEGEO"] = datagf["ENTIDAD"] + datagf["MUN"] + datagf["LOC"] + datagf["AGEB"]

<ipython-input-44-0cbb3daff712>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datagf["CVEGEO"] = datagf["ENTIDAD"] + datagf["MUN"] + datagf["LOC"] + datagf["AGEB"]


In [45]:
poblaciones = datagf[["CVEGEO","POBTOT"]]

In [64]:
agebs["centroide"] = agebs["geometry"].centroid.to_crs("EPSG:4326")

In [66]:
agebs["x"] = agebs["centroide"].x
agebs["y"] = agebs["centroide"].y

In [67]:
agebs

,CVEGEO,CVE_ENT,CVE_MUN,CVE_LOC,CVE_AGEB,geometry,centroide,x,y
0,0901000011716,09,010,0001,1716,"POLYGON ((2787237.541 816989.461, 2787288.728 ...",POINT (-99.26029 19.32199),-99.260288,19.321990
1,0901000012150,09,010,0001,2150,"POLYGON ((2794154.458 823013.444, 2794155.774 ...",POINT (-99.19330 19.37927),-99.193297,19.379268
2,0901000011133,09,010,0001,1133,"POLYGON ((2795690.723 820050.788, 2795684.238 ...",POINT (-99.18462 19.34641),-99.184619,19.346414
3,0901000011307,09,010,0001,1307,"POLYGON ((2792584.475 815678.668, 2792624.325 ...",POINT (-99.21148 19.31029),-99.211482,19.310287
4,0901000010281,09,010,0001,0281,"POLYGON ((2788845.392 823526.074, 2788840.549 ...",POINT (-99.24396 19.38480),-99.243956,19.384795
...,...,...,...,...,...,...,...,...,...
2426,0900700012564,09,007,0001,2564,"POLYGON ((2814016.268 821043.511, 2814019.319 ...",POINT (-99.00418 19.35348),-99.004176,19.353479
2427,0900700012615,09,007,0001,2615,"POLYGON ((2814358.791 820744.850, 2814405.087 ...",POINT (-99.00108 19.35164),-99.001085,19.351638
2428,0900700012969,09,007,0001,2969,"POLYGON ((2815993.470 819777.763, 2816019.848 ...",POINT (-98.98288 19.34340),-98.982882,19.343403
2429,0900700013721,09,007,0001,3721,"POLYGON ((2807966.150 821578.350, 2807941.550 ...",POINT (-99.06319 19.36214),-99.063193,19.362144


In [74]:
salida = pd.merge(agebs[["CVEGEO","x","y"]],poblaciones,how="outer",on="CVEGEO").dropna()

In [77]:
salida.to_csv("../datos/procesados/agebs_cdmx_pob.csv",index=False)

In [78]:
salida[:3]

,CVEGEO,x,y,POBTOT
0,0901000011716,-99.260288,19.321990,7042.0
1,0901000012150,-99.193297,19.379268,4588.0
2,0901000011133,-99.184619,19.346414,2203.0
